### Model

{
  "text": "My mother’s name is Sasha Smith. She likes dogs and pedigree cats.",
  
  "tokens": [
    {"text": "My", "start": 0, "end": 2, "id": 0, "ws": true},
    {"text": "mother", "start": 3, "end": 9, "id": 1, "ws": false},
    {"text": "’s", "start": 9, "end": 11, "id": 2, "ws": true},
    {"text": "name", "start": 12, "end": 16, "id": 3, "ws": true },
    {"text": "is", "start": 17, "end": 19, "id": 4, "ws": true },
    {"text": "Sasha", "start": 20, "end": 25, "id": 5, "ws": true},
    {"text": "Smith", "start": 26, "end": 31, "id": 6, "ws": true},
    {"text": ".", "start": 31, "end": 32, "id": 7, "ws": true, "disabled": true},
    {"text": "She", "start": 33, "end": 36, "id": 8, "ws": true},
    {"text": "likes", "start": 37, "end": 42, "id": 9, "ws": true},
    {"text": "dogs", "start": 43, "end": 47, "id": 10, "ws": true},
    {"text": "and", "start": 48, "end": 51, "id": 11, "ws": true, "disabled": true},
    {"text": "pedigree", "start": 52, "end": 60, "id": 12, "ws": true},
    {"text": "cats", "start": 61, "end": 65, "id": 13, "ws": true},
    {"text": ".", "start": 65, "end": 66, "id": 14, "ws": false, "disabled": true}
  ],
  
  "spans": [
    {"start": 20, "end": 31, "token_start": 5, "token_end": 6, "label": "PERSON"},
    {"start": 43, "end": 47, "token_start": 10, "token_end": 10, "label": "NP"},
    {"start": 52, "end": 65, "token_start": 12, "token_end": 13, "label": "NP"}
  ],

}

In [3]:
import spacy

In [4]:
# for each sentence, create its dictionary with text, tokens and spans keys

In [16]:
def entity_linking(sentences):
    
    name_dict = {"juliet": ["juliet_berg"], "robert":["robert_baratheon"], 
                 "baratheon" : ["robert_baratheon", "joffrey_baratheon"], "kristie": ["kristie"]}
    
    criteria = ["he", "she", "his", "her", "himself", "herself"]
    
    # store sentence id with criteria position(s) (PROPN or criteria list)
    store_ids = {}
    
    nlp = spacy.load("en_core_web_sm")
    
    # labels to feed in Prodigy
    labels = []
    
    # for each sentence with criteria, store idx and positions
    for idx, sentence in enumerate(sentences) :
        
        # store word position with criteria
        store_position = []

        sentence = nlp(sentence)

        for token in sentence:
            
            # check if it is a PRON or DET
            if token.pos_ == "PRON" or token.pos_ == "DET":

                # check if it is a 3rd person
                if str(token).lower() in criteria :
                    store_position.append(str(sentence).split().index(str(token)))
            
            # check if it is a name    
            if token.pos_ == "PROPN": 
                store_position.append(str(sentence).split().index(str(token)))
      
        store_ids[idx] = store_position
                
    # process sentences for prodigy
    for i,j in store_ids.items():
        print(i, j)

        to_return = {"text":''}

        # tokens displayed in Prodigy
        tokens = []

        # spans for EL
        spans = []

        # character counter in the text
        start = 0

        # word id
        ids = 0
        """
        # check if previous sentence exists (context)
        if i != 0:
            previous = sentences[i -1]
        else:
            previous = None

        # check if next sentence exists (context)
        if i != len(sentences):
            nexts = sentences[i + 1]
        else:
            nexts = None"""

        #if previous != None and nexts != None:
        if j:
            #print(sentences[i])
            to_return['text'] += sentences[i]
            print(to_return['text'])

            # get tokens
            for word in sentences[i].split(' '):
                # do no add +1 to start counter for last word
                if "\n" in word : 
                    tokens.append({"text": str(word), "start": start , "end": len(str(word))+start-1, "id": ids, "ws": True})
                    
                else : 
                    tokens.append({"text": str(word), "start": start , "end": len(str(word))+start, "id": ids, "ws": True})
                    start+= len(str(word))+1
                # change start token for next word
                ids+=1

            to_return["tokens"] = tokens
            
            # get spans
            for position in j:
                for dic in tokens:
                    if position == dic["id"]:
                        print("FIND TOKEN",dic["text"])
                        print(dic["start"], dic["end"])
                        corresponding_token = to_return["text"][dic["start"]:dic["end"]].lower()
                        spans.append({"start" : dic["start"], "end" : dic["end"], "token_start": position, "token_end":position, "label": "DETECT"})
                
                        # find corresponding labels
                        if corresponding_token in name_dict:
                            print("IN DIC NAME", corresponding_token)
                            
                            for name in name_dict[corresponding_token]:
                                labels.append(name)
            
            to_return["labels"] = labels
            to_return["spans"] = spans

            #print(to_return)
            yield to_return
"""
    # refresh counters
    start = 0    
    ids = 0
    
    print({to_return["text"]})
    # append EL span to spans
    for sentence in sentences[slices:slices+3]:
        
        sentence = nlp(sentence)
        print(sentence)

        # get EL
        for word in sentence:
            
            if word.pos_ == "PRON":
                spans.append({"start": start, "end": start + len(str(word)), "token_start": ids, "token_end": ids, "label": "PRON", "word":word})
                start+= len(str(word)) + 1
                ids+=1
                
            if word.pos_ == "PROPN":
                spans.append({"start": start, "end": start + len(str(word)), "token_start": ids, "token_end": ids, "label": "PERSON","word":word})
                start+= len(str(word)) + 1
                ids+=1

            if word.pos_ == "DET":
                spans.append({"start": start, "end": start + len(str(word)), "token_start": ids, "token_end": ids, "label": "DET","word":word})
                start+= len(str(word)) + 1
                ids+=1
            else:
                ids+=1
"""               
    

'\n    # refresh counters\n    start = 0    \n    ids = 0\n    \n    print({to_return["text"]})\n    # append EL span to spans\n    for sentence in sentences[slices:slices+3]:\n        \n        sentence = nlp(sentence)\n        print(sentence)\n\n        # get EL\n        for word in sentence:\n            \n            if word.pos_ == "PRON":\n                spans.append({"start": start, "end": start + len(str(word)), "token_start": ids, "token_end": ids, "label": "PRON", "word":word})\n                start+= len(str(word)) + 1\n                ids+=1\n                \n            if word.pos_ == "PROPN":\n                spans.append({"start": start, "end": start + len(str(word)), "token_start": ids, "token_end": ids, "label": "PERSON","word":word})\n                start+= len(str(word)) + 1\n                ids+=1\n\n            if word.pos_ == "DET":\n                spans.append({"start": start, "end": start + len(str(word)), "token_start": ids, "token_end": ids, "label": "DE

In [22]:
import prodigy
from prodigy.components.preprocess import add_tokens, split_sentences
from plumcot_prodigy.custom_loaders import *
import requests
from plumcot_prodigy.video import mkv_to_base64
from typing import Dict, List, Text, Union
from typing_extensions import Literal
import spacy
from pathlib import Path

# path to shows directories
PATH = "/vol/work1/bergoend/pyannote-db-plumcot/Plumcot/data"

def entity_linking():
    
    # load episode
    episodes_list = load_episodes(PATH)
    #episodes_list = ["BuffyTheVampireSlayer.Season01.Episode12"]
    
    for episode in episodes_list:
        print("\nCurrent Episode :", episode)
        
        # process serie or film
        if len(episode.split('.')) == 3:
            series, _, _ = episode.split('.')
        elif len(episode.split('.')) == 2:
            series, _ = episode.split('.')
            
        # load mkv, aligned file & aligned sentences
        mkv, aligned, sentences = load_files(series, episode, PATH)

        #######sentences = ["In the name of The King Robert Baratheon", "My name is Juliet\n", "The cat is eating", "My cat is my best friend, he is kind\n", "The frog is eating", "Her name is Kristie\n", "The wolf is eating"]

        #######name_dict = {"juliet": ["juliet_berg"], "robert":["robert_baratheon"], 
                     #"baratheon" : ["robert_baratheon", "joffrey_baratheon"], "kristie": ["kristie"]}

        criteria = ["he", "she", "his", "her", "himself", "herself"]

        # store sentence id with criteria position(s) (PROPN or criteria list)
        store_ids = {}

        # labels to feed in Prodigy
        labels = []

        # for each sentence with criteria, store idx and positions
        for idx, sentence in enumerate(sentences) :

            # store word position with criteria
            store_position = []

            for token in sentence:
                #print(token, token.pos_)
                # check if it is a PRON or DET
                if token.pos_ == "PRON" or token.pos_ == "DET":

                    # check if it is a 3rd person
                    if str(token).lower() in criteria :
                        
                        # check if entity_linking field is empty
                        if token._.entity_linking == "_" or token._.entity_linking == "?":
                            #print("\tFOUND PRON OR DET")
                            store_position.append(str(sentence).split().index(str(token)))

                # check if it is a name    
                if token.pos_ == "PROPN":                     
                    if str(token) != "ve" and str(token) != "m" and str(token)[0].isupper():
                    
                        # check if entity_linking field is empty
                        if token._.entity_linking == "_" or token._.entity_linking == "?":
                            #print("\tFOUND PROP NOUN")
                            store_position.append(str(sentence).split().index(str(token)))

            # keep sentence id, tokens positions, sentence id
            store_ids[idx] = (store_position, sentence)

        # process sentences for prodigy
        for i,j in store_ids.items():

            to_return = {"text":''}

            # tokens displayed in Prodigy
            tokens = []

            # spans for EL
            spans = []

            # character counter in the text
            start = 0

            # word id
            ids = 0
            
            # check if previous sentence exists (context)
            if i != 0:
                previous = sentences[i -1]
            else:
                previous = None

            # check if next sentence exists (context)
            if i != len(sentences)-1:
                nexts = sentences[i + 1]
            else:
                nexts = None

            if previous != None and nexts != None:
                if j[0]:
                    
                    #print(sentences[i])
                    to_return['text'] = f"{str(previous)} {str(sentences[i])} {str(nexts)}"
                    print(to_return['text'])
                    
                    tokens.append({"text": str(previous), "start": start , "end": len(str(previous))+start, "id": ids, "ws": False})
                    start +=len(str(previous))+1
                    #print(start)
                    #print(len(str(previous)))
                    ids +=1

                    # get tokens
                    for word in str(sentences[i]).split(' '):
                        # do no add +1 to start counter for last word
                        if "\n" in word : 
                            tokens.append({"text": str(word), "start": start , "end": len(str(word))+start-1, "id": ids, "ws": True})

                        else : 
                            tokens.append({"text": str(word), "start": start , "end": len(str(word))+start, "id": ids, "ws": True})
                            start+= len(str(word))+1
                        # change start token for next word
                        ids+=1
                    start+=1    
                    tokens.append({"text": str(nexts), "start": start-1 , "end": len(str(nexts))+start-1, "id": ids, "ws": False})
                    
            print(tokens)
            to_return["tokens"] = tokens
            
            for i in to_return["tokens"]:
                print(to_return["text"][i["start"]: i["end"]])

In [23]:
sentences = ["In the name of The King Robert Baratheon\n", "My name is Juliet\n", "The cat is eating\n", "My cat is my best friend, he is kind\n", "The frog is eating\n", "Her name is Kristie\n", "The wolf is eating\n"]
el = entity_linking()

for dictionary in el:
    #print(dictionary, '\n')
    text = dictionary["text"]
    print(dictionary["labels"])
    
    #for tok in dictionary['tokens']:
        #print(text[tok['start']: tok["end"]])
        


Current Episode : Friends.Season01.Episode01
[]
There 's nothing to tell ! He 's just some guy I work with ! C'm on , you 're going out with the guy !
[{'text': "There 's nothing to tell !", 'start': 0, 'end': 26, 'id': 0, 'ws': False}, {'text': 'He', 'start': 27, 'end': 29, 'id': 1, 'ws': True}, {'text': "'s", 'start': 30, 'end': 32, 'id': 2, 'ws': True}, {'text': 'just', 'start': 33, 'end': 37, 'id': 3, 'ws': True}, {'text': 'some', 'start': 38, 'end': 42, 'id': 4, 'ws': True}, {'text': 'guy', 'start': 43, 'end': 46, 'id': 5, 'ws': True}, {'text': 'I', 'start': 47, 'end': 48, 'id': 6, 'ws': True}, {'text': 'work', 'start': 49, 'end': 53, 'id': 7, 'ws': True}, {'text': 'with', 'start': 54, 'end': 58, 'id': 8, 'ws': True}, {'text': '!', 'start': 59, 'end': 60, 'id': 9, 'ws': True}, {'text': "C'm on , you 're going out with the guy !", 'start': 61, 'end': 102, 'id': 10, 'ws': False}]
There 's nothing to tell !
He
's
just
some
guy
I
work
with
!
C'm on , you 're going out with the guy !


[]
[]
[]
[]
[]
[]
[]
[]
Yeah , I think for us , kissing is pretty much like an opening act , y'know ? I mean it 's like the stand - up comedian you have to sit through before Pink Floyd comes out . Yeah , and - and it 's not that we do n't like the comedian
[{'text': "Yeah , I think for us , kissing is pretty much like an opening act , y'know ?", 'start': 0, 'end': 77, 'id': 0, 'ws': False}, {'text': 'I', 'start': 78, 'end': 79, 'id': 1, 'ws': True}, {'text': 'mean', 'start': 80, 'end': 84, 'id': 2, 'ws': True}, {'text': 'it', 'start': 85, 'end': 87, 'id': 3, 'ws': True}, {'text': "'s", 'start': 88, 'end': 90, 'id': 4, 'ws': True}, {'text': 'like', 'start': 91, 'end': 95, 'id': 5, 'ws': True}, {'text': 'the', 'start': 96, 'end': 99, 'id': 6, 'ws': True}, {'text': 'stand', 'start': 100, 'end': 105, 'id': 7, 'ws': True}, {'text': '-', 'start': 106, 'end': 107, 'id': 8, 'ws': True}, {'text': 'up', 'start': 108, 'end': 110, 'id': 9, 'ws': True}, {'text': 'comedian', 'start': 111, 'end': 11

[]
[]
[]
[]
[]
[]
Um , not so good . He walked me to the subway and said ' We should do this again ! ' Ohh .
[{'text': 'Um , not so good .', 'start': 0, 'end': 18, 'id': 0, 'ws': False}, {'text': 'He', 'start': 19, 'end': 21, 'id': 1, 'ws': True}, {'text': 'walked', 'start': 22, 'end': 28, 'id': 2, 'ws': True}, {'text': 'me', 'start': 29, 'end': 31, 'id': 3, 'ws': True}, {'text': 'to', 'start': 32, 'end': 34, 'id': 4, 'ws': True}, {'text': 'the', 'start': 35, 'end': 38, 'id': 5, 'ws': True}, {'text': 'subway', 'start': 39, 'end': 45, 'id': 6, 'ws': True}, {'text': 'and', 'start': 46, 'end': 49, 'id': 7, 'ws': True}, {'text': 'said', 'start': 50, 'end': 54, 'id': 8, 'ws': True}, {'text': "'", 'start': 55, 'end': 56, 'id': 9, 'ws': True}, {'text': 'We', 'start': 57, 'end': 59, 'id': 10, 'ws': True}, {'text': 'should', 'start': 60, 'end': 66, 'id': 11, 'ws': True}, {'text': 'do', 'start': 67, 'end': 69, 'id': 12, 'ws': True}, {'text': 'this', 'start': 70, 'end': 74, 'id': 13, 'ws': True},

[]
Alright . Phoebe ? Okay , okay .
[{'text': 'Alright .', 'start': 0, 'end': 9, 'id': 0, 'ws': False}, {'text': 'Phoebe', 'start': 10, 'end': 16, 'id': 1, 'ws': True}, {'text': '?', 'start': 17, 'end': 18, 'id': 2, 'ws': True}, {'text': 'Okay , okay .', 'start': 19, 'end': 32, 'id': 3, 'ws': False}]
Alright .
Phoebe
?
Okay , okay .
[]
[]
[]
[]
[]
you took mine . Chandler , what about you ? Uh , if I were omnipotent for a day , I 'd . .
[{'text': 'you took mine .', 'start': 0, 'end': 15, 'id': 0, 'ws': False}, {'text': 'Chandler', 'start': 16, 'end': 24, 'id': 1, 'ws': True}, {'text': ',', 'start': 25, 'end': 26, 'id': 2, 'ws': True}, {'text': 'what', 'start': 27, 'end': 31, 'id': 3, 'ws': True}, {'text': 'about', 'start': 32, 'end': 37, 'id': 4, 'ws': True}, {'text': 'you', 'start': 38, 'end': 41, 'id': 5, 'ws': True}, {'text': '?', 'start': 42, 'end': 43, 'id': 6, 'ws': True}, {'text': "Uh , if I were omnipotent for a day , I 'd . .", 'start': 44, 'end': 90, 'id': 7, 'ws': False}]
yo

ValueError: not enough values to unpack (expected 8, got 7)